In [1]:
#Importing all modules
import sys
import time
import os

import pandas as pd

from loguru import logger
from pathlib import Path

from Naive.node import Node

In [2]:
time_series_data = pd.read_csv('Dataset/Sales_Transaction_Dataset_Weekly_Final.csv')
time_series_data


,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,P1,11,12,10,8,13,12,14,21,6,...,4,7,8,10,12,3,7,6,5,10
1,P2,7,6,3,2,7,1,6,3,3,...,2,4,5,1,1,4,5,1,6,0
2,P3,7,11,8,9,10,8,7,13,12,...,6,14,5,5,7,8,14,8,8,7
3,P4,12,8,13,5,9,6,9,13,13,...,9,10,3,4,6,8,14,8,7,8
4,P5,8,5,13,11,6,7,9,14,9,...,7,11,7,12,6,6,5,11,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,P815,0,0,1,0,0,2,1,0,0,...,0,1,1,0,0,1,0,0,2,0
807,P816,0,1,0,0,1,2,2,6,0,...,3,3,4,2,4,5,5,5,6,5
808,P817,1,0,0,0,1,1,2,1,1,...,2,0,0,2,2,0,0,0,4,3
809,P818,0,0,0,1,0,0,0,0,1,...,0,0,0,1,1,0,0,0,2,0


In [3]:
columns = list(time_series_data.columns)
time_series_index = columns.pop(0)  # remove product code

time_series_dict = dict()

for index, row in time_series_data.iterrows():
    time_series_dict[row[time_series_index]] = list(row[columns])

time_series_dict

{'P1': [11,
  12,
  10,
  8,
  13,
  12,
  14,
  21,
  6,
  14,
  11,
  14,
  16,
  9,
  9,
  9,
  14,
  9,
  3,
  12,
  5,
  11,
  7,
  12,
  5,
  9,
  7,
  10,
  5,
  11,
  7,
  10,
  12,
  6,
  5,
  14,
  10,
  9,
  12,
  17,
  7,
  11,
  4,
  7,
  8,
  10,
  12,
  3,
  7,
  6,
  5,
  10],
 'P2': [7,
  6,
  3,
  2,
  7,
  1,
  6,
  3,
  3,
  3,
  2,
  2,
  6,
  2,
  0,
  6,
  2,
  7,
  7,
  9,
  4,
  7,
  2,
  4,
  5,
  3,
  5,
  8,
  5,
  5,
  3,
  1,
  3,
  2,
  3,
  10,
  5,
  2,
  7,
  3,
  2,
  5,
  2,
  4,
  5,
  1,
  1,
  4,
  5,
  1,
  6,
  0],
 'P3': [7,
  11,
  8,
  9,
  10,
  8,
  7,
  13,
  12,
  6,
  14,
  9,
  4,
  7,
  12,
  8,
  7,
  11,
  10,
  7,
  7,
  13,
  11,
  8,
  10,
  8,
  14,
  5,
  3,
  13,
  11,
  9,
  7,
  8,
  7,
  9,
  6,
  12,
  12,
  9,
  3,
  5,
  6,
  14,
  5,
  5,
  7,
  8,
  14,
  8,
  8,
  7],
 'P4': [12,
  8,
  13,
  5,
  9,
  6,
  9,
  13,
  13,
  11,
  8,
  4,
  5,
  4,
  15,
  7,
  11,
  9,
  15,
  4,
  6,
  7,
  11,
  7,
  9,
  6,
  10,
  

In [4]:
def create_tree_phase(time_series_data, P_value, paa_value,max_level):
    
    good_leaf_nodes = list()
    bad_leaf_nodes = list()

    print("Create-tree phase: start node splitting")
    node = Node(level=1, group=time_series_data, paa_value=5)
    node.start_splitting(P_value, max_level, good_leaf_nodes, bad_leaf_nodes) # using naive method node splitting 
    
    return good_leaf_nodes, bad_leaf_nodes
    

In [5]:
def recycle_bad_leaves_phase(p_value,good_leaf_nodes, bad_leaf_nodes,paa_value):
    
    suppressed_nodes = list()
    bad_leaves_node_dict = dict()
        
    for node in bad_leaf_nodes:
        if node.level in bad_leaves_node_dict.keys():
            bad_leaves_node_dict[node.level].append(node)
        else:
            bad_leaves_node_dict[node.level] = [node]

    bad_leaf_nodes_size = sum([node.size for node in bad_leaf_nodes])
    
        
    if bad_leaf_nodes_size >= p_value:
        
        current_level = max(bad_leaves_node_dict.keys())
        
        while bad_leaf_nodes_size >= p_value:
            
            if current_level in bad_leaves_node_dict.keys():
                leave_merge_dict = dict()
                keys_remove_list = list()
                merge = False
                
                for current_level_node in bad_leaves_node_dict[current_level]:
                    pattern_rep_node = current_level_node.pattern_representation
                    if pattern_rep_node in leave_merge_dict.keys():
                        merge = True
                        leave_merge_dict[pattern_rep_node].append(current_level_node)
                        if pattern_rep_node in keys_remove_list:
                            keys_remove_list.remove(pattern_rep_node)
                        else:
                            leave_merge_dict[pattern_rep_node] = [current_level_node]
                            keys_remove_list.append(pattern_rep_node)
                    
                    if merge:
                        for k in keys_remove_list:
                            del leave_merge_dict[k]

                        for pr, node_list in leave_merge_dict.items():
                            group = dict()
                            for node in node_list:
                                bad_leaves_node_dict[current_level].remove(node)
                                group.update(node.group)
                            if current_level > 1:
                                level = current_level
                            else:
                                level = 1
                            leaf_merge = Node(level=level, pattern_representation=pr,
                                group=group, paa_value=paa_value)

                            if leaf_merge.size >= p_value:
                                leaf_merge.label = "good-leaf"
                                good_leaf_nodes.append(leaf_merge)
                                bad_leaf_nodes_size -= leaf_merge.size
                            else: 
                                leaf_merge.label = "bad-leaf"
                                bad_leaves_node_dict[current_level].append(leaf_merge)

                new_level = current_level-1
                for node in bad_leaves_node_dict[current_level]:
                    if new_level > 1:
                        values_group = list(node.group.values())
                        data = np.array(values_group[0])
                        data_znorm = znorm(data)
                        data_paa = paa(data_znorm, paa_value)
                        pr = ts_to_string(data_paa, cuts_for_asize(new_level))
                    else:
                        pr = "a"*paa_value
                    node.level = new_level
                    node.pattern_representation = pr

                if current_level > 0:
                    if new_level not in bad_leaves_node_dict.keys():
                        bad_leaves_node_dict[new_level] = bad_leaves_node_dict.pop(current_level)
                    else:
                        bad_leaves_node_dict[new_level] = bad_leaves_node_dict[new_level] + bad_leaves_node_dict.pop(current_level) 
                    current_level -= 1
                else:
                    break 

        
        remaining_bad_leaf_nodes = list(bad_leaves_node_dict.values())[0]
        for node in remaining_bad_leaf_nodes:
            suppressed_nodes.append(node)
        print (suppressed_nodes)

    return suppressed_nodes

In [ ]:
def group_formation_phase(p_subgroups, p, k, kgroup_list):
 
    P_group_list = list() 
    splitted_p_subgroups = list()
    p_subgroup_split_indexes = list()
    
    print("Start group formation phase")
    
    # Initialize P_group_list 
    for p_subgroup in p_subgroups: 
        P_group_list.append(p_subgroup)

  
    for p_subgroup_index, p_subgroup in enumerate(P_group_list): 

        if len(p_subgroup) >= 2*p:
        
            tree_clustering = list()
            temp_splitted_p_subgroup = list()

            p_subgroup_to_be_splitted = p_subgroup.copy()
            top_down_greedy_clustering( p_subgroup_to_be_splitted, p, temp_splitted_p_subgroup, tree_clustering)
 
            postprocessed_p_subgroups = list()
            top_down_greedy_clustering_postprocessing(p,temp_splitted_p_subgroup,tree_clustering,postprocessed_p_subgroups) 
                                                            
            splitted_p_subgroups += postprocessed_p_subgroups
            p_subgroup_split_indexes.append(p_subgroup_index) 
    
    P_group_list = [p_subgroup for (p_subgroup_index, p_subgroup) in enumerate(P_group_list) if p_subgroup_index not in p_subgroup_split_indexes]
    P_group_list += splitted_p_subgroups
    
    p_subgroups_k_promoted_idxs = list() 

    for p_subgroup_index, p_subgroup in enumerate(P_group_list):
        
        if len(p_subgroup) >= k:
            p_subgroups_k_promoted_idxs.append(p_subgroup_index)
            kgroup_list.append(p_subgroup)

    
    P_group_list = [p_subgroup for (p_subgroup_index, p_subgroup) in enumerate(P_group_list) if p_subgroup_index not in p_subgroups_k_promoted_idxs]

    p_subgroups_index_merged = list()
    P_group_list_size= sum([len(p_subgroup) for p_subgroup in P_group_list])

  
    while P_group_list_size>= k:
     
        Group, group_index = group_min_value_loss(group_to_search=P_group_list,index_ignored=p_subgroups_index_merged)
        p_subgroups_index_merged.append(group_index) 
        P_group_list_size-= len(Group)

        while len(Group) < k:
           
            group_min, group_min_index = group_min_value_loss(P_group_list,Group,p_subgroups_index_merged)
            p_subgroups_index_merged.append(group_min_index)
            
            Group.update(group_min) 
            P_group_list_size-= len(group_min)
       
        kgroup_list.append(Group) 


    p_subgroups_left = [p_subgroup for (p_subgroup_index, p_subgroup) in enumerate(P_group_list) if p_subgroup_index not in p_subgroups_index_merged]

    # for each remaining p-subgroup
    for p_subgroup in p_subgroups_left:
       
        k_group_remaining, k_group_remaining_idx = group_min_value_loss(kgroup_list,p_subgroup)   
        kgroup_list.pop(k_group_remaining_idx)
        
        k_group_remaining.update(p_subgroup)
        kgroup_list.append(k_group_remaining)
        
    print("End group formation phase")
    

    
    

In [6]:
def KAPRA(K_value, P_value, paa_value ,max_level,time_series_data):
    
    #Create tree phase
    good_leaf_nodes, bad_leaf_nodes = create_tree_phase(time_series_data, P_value, paa_value,max_level)
    print ('Good leaf nodes :' + str (len (good_leaf_nodes)) + ' Bad leaf nodes :' + str (len (bad_leaf_nodes)) )
    
    #Recycle bad-leaves phase
    if(len(bad_leaf_nodes) > 0):
        print ('recyling bad leaves phases')
        suppressed_nodes = recycle_bad_leaves_phase(P_value, good_leaf_nodes, bad_leaf_nodes, paa_value)
    
    print(suppressed_nodes)
    suppressed_group_list =list()
    for node in suppressed_nodes:
        suppressed_group_list.append(node.group)
    
    print(suppressed_group_list)
    
    #group formation phase
    pattern_representation_list, k_group_list = group_formation_phase(P_subgroups, P_value)

    #Anonymize data
    #anonymize_data(pattern_representation_list,k_group_list,suppressed_nodes)
    

K_value = 10
P_value = 2
paa_value = 5
max_level = 4
time_series_data = time_series_dict

KAPRA(K_value, P_value, paa_value,max_level, time_series_data)

Create-tree phase: start node splitting


2022-04-22 16:10:16.181 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-22 16:10:16.182 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-22 16:10:16.184 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 1 as bad-leaf and split only tg_nodes 29
2022-04-22 16:10:16.218 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-22 16:10:16.220 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-22 16:10:16.221 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 1 as bad-leaf and split only tg_nodes 2
2022-04-22 16:10:16.252 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-22 16:10:16.254 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-22 16:10:16.255 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 0 as bad-leaf and split only tg_nodes

2022-04-22 16:10:17.459 | INFO     | Naive.node:start_splitting:125 - Merge all bad nodes in a single node, and label it as good-leaf
2022-04-22 16:10:17.461 | INFO     | Naive.node:start_splitting:136 - Split only tg_nodes 2
2022-04-22 16:10:17.463 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:2, p_value:2 == good-leaf
2022-04-22 16:10:17.476 | INFO     | Naive.node:maximize_level_node:228 - New level for node: 4
2022-04-22 16:10:17.480 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:2, p_value:2 == good-leaf
2022-04-22 16:10:17.501 | INFO     | Naive.node:maximize_level_node:228 - New level for node: 4
2022-04-22 16:10:17.529 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-22 16:10:17.530 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-22 16:10:17.534 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 0 as bad-leaf and split only tg_nodes 3
2022-04-22 16:10

2022-04-22 16:10:18.241 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-22 16:10:18.242 | INFO     | Naive.node:start_splitting:125 - Merge all bad nodes in a single node, and label it as good-leaf
2022-04-22 16:10:18.243 | INFO     | Naive.node:start_splitting:136 - Split only tg_nodes 4
2022-04-22 16:10:18.244 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:2, p_value:2 == good-leaf
2022-04-22 16:10:18.260 | INFO     | Naive.node:maximize_level_node:228 - New level for node: 4
2022-04-22 16:10:18.266 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:2, p_value:2 == good-leaf
2022-04-22 16:10:18.276 | INFO     | Naive.node:maximize_level_node:228 - New level for node: 4
2022-04-22 16:10:18.281 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:2, p_value:2 == good-leaf
2022-04-22 16:10:18.293 | INFO     | Naive.node:maximize_level_node:228 - New level for node: 4
2022-04-22 16:10

2022-04-22 16:10:19.095 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 1 as bad-leaf and split only tg_nodes 2
2022-04-22 16:10:19.097 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:3, p_value:2 == good-leaf
2022-04-22 16:10:19.117 | INFO     | Naive.node:maximize_level_node:228 - New level for node: 4
2022-04-22 16:10:19.122 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:2, p_value:2 == good-leaf
2022-04-22 16:10:19.136 | INFO     | Naive.node:maximize_level_node:228 - New level for node: 4
2022-04-22 16:10:19.158 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-22 16:10:19.159 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-22 16:10:19.160 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 0 as bad-leaf and split only tg_nodes 2
2022-04-22 16:10:19.176 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-22 16:10:19.177 | INFO

Create-tree phase: finish node splitting
Good leaf nodes :78 Bad leaf nodes :9
recyling bad leaves phases
[<Naive.node.Node object at 0x000001BD450DA4F0>, <Naive.node.Node object at 0x000001BD450DA310>, <Naive.node.Node object at 0x000001BD450D8D30>, <Naive.node.Node object at 0x000001BD450C5D60>, <Naive.node.Node object at 0x000001BD45104640>, <Naive.node.Node object at 0x000001BD45104910>, <Naive.node.Node object at 0x000001BD45104130>, <Naive.node.Node object at 0x000001BD4510D370>, <Naive.node.Node object at 0x000001BD4510D6A0>]
[<Naive.node.Node object at 0x000001BD450DA4F0>, <Naive.node.Node object at 0x000001BD450DA310>, <Naive.node.Node object at 0x000001BD450D8D30>, <Naive.node.Node object at 0x000001BD450C5D60>, <Naive.node.Node object at 0x000001BD45104640>, <Naive.node.Node object at 0x000001BD45104910>, <Naive.node.Node object at 0x000001BD45104130>, <Naive.node.Node object at 0x000001BD4510D370>, <Naive.node.Node object at 0x000001BD4510D6A0>]
[{'P677': [0, 1, 0, 0, 0, 0,